# PHY408 Final Report
Ben Pennell

April 26th, 2023

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from astropy.visualization import ImageNormalize, AsinhStretch
from astropy.convolution import convolve, Gaussian2DKernel
from photutils import detect_sources
from astropy.stats import SigmaClip
from photutils.background import Background2D, SExtractorBackground
from scipy import stats as st
from photutils.segmentation import SourceCatalog

FITS_PATH = "./A2390C.fits"
hdul = fits.open(FITS_PATH, memmap=False)
data = hdul[0].data
header = hdul[0].header
hdul.close()

SUBTRACTED_PATH = "./cube_subtracted.fits"
SEGM_PATH = "./segmentation_map.fits"
CUBE_LPF_PATH = "./cube_lpf.fits"
DEEP_FRAME_PATH = "./cube_deep_frame.fits"
START_INDEX = 42
END_INDEX = 88

In [ ]:
def display_fits(image, size=7):
    plt.rcParams["figure.figsize"] = [size, size]
    plt.rcParams["figure.autolayout"] = True
    
    norm = ImageNormalize(vmin=np.median(image), vmax=np.median(image) + 3*np.std(image), stretch=AsinhStretch(a=0.1))

    ax = plt.gca()
    ax.imshow(image, origin="lower", norm=norm)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
# get wavenumber array
min_wavenumber = header['CRVAL3']
wavenumber_step = header['CDELT3']
step_count = header['NAXIS3']
max_wavenumber = min_wavenumber+(step_count-1)*wavenumber_step
wavn_array = np.arange(min_wavenumber, max_wavenumber, wavenumber_step)
wavn_array = wavn_array[START_INDEX:END_INDEX]

## Remove Fringes

### Source Detection

In [ ]:
def segm_map(image, SN=5, npixels=5, sigma=3, size=128, return_all=False):
    sigma_clip = SigmaClip(sigma=sigma)
    bkg = Background2D(np.nan_to_num(image), (size, size), filter_size=(3, 3),
                sigma_clip=sigma_clip, bkg_estimator=SExtractorBackground())
    
    threshold = bkg.background + (SN * bkg.background_rms)

    sources = detect_sources(image, threshold, npixels=npixels)
    
    if return_all:
        return sources
    else:
        return sources.data

### Background Subtraction

In [ ]:
def subtract_background(image, sigma=3, size=128, SN=5, mask=None, iterations=1):
    temp_image = image
    for _ in range(iterations):
        sigma_clip = SigmaClip(sigma=sigma)
        mask = (segm_map(image, SN=SN) != 0)
        bkg = Background2D(np.nan_to_num(image), (size, size), filter_size=(3, 3),
                    sigma_clip=sigma_clip, bkg_estimator=SExtractorBackground(), coverage_mask=mask)
        temp_image = temp_image - bkg.background
    return temp_image

### Clean data and create deep frame, segmentation map

In [ ]:
# subtract backgrounds
fixed_data = []
for image in data:
    image = image[50:-50,50:-50]
    fixed_data.append(subtract_background(image))
fixed_data = np.array(fixed_data)

# collapse all images into one image by taking the max at each pixel
deep_frame_initial = np.max(data[START_INDEX:END_INDEX,50:-50,50:-50], axis=0)

# create the initial segmentation map - map of all detections
segmentation_map_initial = segm_map(deep_frame_initial)

# save background subtracted cube
hdu = fits.PrimaryHDU(fixed_data)
hdulist = fits.HDUList([hdu])
hdulist.writeto(SUBTRACTED_PATH, overwrite=True)

### Low-Pass Filter

In [ ]:
cube_lpf = []
for image in fixed_data[START_INDEX:END_INDEX]:
    # set all values containing detected ELGs to 0
    masked_image = np.where(segmentation_map_initial != 0, 0, image)

    working_image = masked_image

    # Gaussian kernel elongated in x direction
    gaussian = Gaussian2DKernel(12, 3)

    # Convolve with masked image
    convolved_image = convolve(working_image, gaussian, 
                            mask=None, nan_treatment="fill", normalize_kernel=True)

    # subtract this high frequency image to only allow low frequencies
    final_image = image - convolved_image
    cube_lpf.append(final_image)

### Prepare for cross-correlation

In [ ]:
# save lpf
hdu = fits.PrimaryHDU(cube_lpf)
hdulist = fits.HDUList([hdu])
hdulist.writeto(CUBE_LPF_PATH, overwrite=True)

# save deep frame
deep_frame_final = np.max(cube_lpf, axis=0)
hdu = fits.PrimaryHDU(deep_frame_final)
hdulist = fits.HDUList([hdu])
hdulist.writeto(DEEP_FRAME_PATH, overwrite=True)

# save segmentation map
segm_map_final = segm_map(deep_frame_final)

hdu = fits.PrimaryHDU(segm_map_final)
hdulist = fits.HDUList([hdu])
hdulist.writeto(SEGM_PATH, overwrite=True)

segmentation_map_initial = segm_map(deep_frame_initial)

## Cross-Correlation

In [ ]:
Ha_line = 6562.819
NII_u = 6583.460
NII_l = 6548.050

Ha_lower_ratio = 4
Ha_upper_ratio = 12
Ha_size = 1

field_z = 0.228

In [ ]:
def gaussian(t, t_h):
    amp = 1 / (np.sqrt(np.pi) * t_h)
    exponent = - (t / t_h)**2
    return amp * np.exp(exponent)

### The Template

In [ ]:
size = ((1e8)/wavn_array[0] - (1e8)/wavn_array[len(wavn_array) - 1])/2
wavl_gauss = np.arange(Ha_line - size, Ha_line + size, 1)

index_NIIu = np.argmin(np.abs(NII_u - wavl_gauss))
index_NIIl = np.argmin(np.abs(NII_l - wavl_gauss))
index_ha = np.argmin(np.abs(Ha_line - wavl_gauss))

template = np.zeros(len(wavl_gauss))
template[index_ha] = Ha_size
template[index_NIIu] = Ha_size / Ha_upper_ratio
template[index_NIIl] = Ha_size / Ha_lower_ratio

convolution_gaussian = []
t_h = 3
for t in np.arange(0, len(template), 1) - (0.5 * len(template)):
    convolution_gaussian.append(gaussian(t, t_h))

convolved_template = np.convolve(template, convolution_gaussian, mode="same")
wavl_gauss_shifted = wavl_gauss * (1 + field_z)

### Cross Correlation

In [ ]:
def pad_zeroes(series, number=None, both=False):
    count = 0
    if number is None:
        power = np.ceil(np.log2(len(series)))
        count = 2**power - len(series)
    else:
        count = number

    if not both:
        padded_series = np.concatenate((series, np.zeros(int(count))))
        return padded_series
    else:
        padded_series = np.concatenate((np.zeros(int(count / 2)), series))
        padded_series = np.concatenate((padded_series, np.zeros(int(count / 2))))
        return padded_series
    
def normalize(x):
    return (x-np.min(x))/(np.max(x)-np.min(x))

In [ ]:
def determine_quality(cc):
    return np.max(cc) / np.mean(abs(cc))

def cross_correlate(x_centroid, y_centroid):
    spectrum = fixed_data[:, int(y_centroid), int(x_centroid)]
    spectrum = np.flip(spectrum[START_INDEX:END_INDEX])
    wavelengths = np.linspace((1e8)/wavn_array[len(wavn_array) - 1], (1e8)/wavn_array[0], len(spectrum))

    new_spectrum = np.interp(wavl_gauss_shifted, wavelengths, spectrum)

    c_template = pad_zeroes(convolved_template)
    c_spectrum = pad_zeroes(np.nan_to_num(new_spectrum))

    cross_correlation = np.correlate(c_spectrum, c_template, mode="same")

    frequencies = np.fft.fftfreq(len(c_spectrum), 1) * len(c_spectrum)
    lag_times = np.fft.fftshift(frequencies)

    offset = np.argmax(cross_correlation)
    offset = lag_times[offset]

    observed = wavl_gauss_shifted[np.argmax(convolved_template)] + offset
    redshift = (observed)/(Ha_line) - 1

    sn = determine_quality(cross_correlation)

    return (wavelengths, spectrum), offset, redshift, sn

def create_catalogue(sources):
    outdata = "NAME X Y KRON_FLUX REDSHIFT QUALITY"
    gooddata = "NAME X Y KRON_FLUX REDSHIFT QUALITY"

    for i, source in enumerate(sources):
        x_centroid = source.centroid[0]
        y_centroid = source.centroid[1]
        kron_flux = source.kron_flux

        spectra, offset, redshift, sn = cross_correlate(x_centroid, y_centroid)

        line = "\n" + str(i) + " " + str(round(x_centroid, 2)) + " " + str(round(y_centroid, 2)) + " " + str(round(kron_flux, 2)) + " " + str(round(redshift, 4)) + " " + str(round(sn, 2))
        outdata += line
        if sn > 4:
            gooddata += line

    outfile = open("all_objects.txt", "w")
    outfile.write(outdata)

    goodfile = open("catalogue.txt", "w")
    goodfile.write(gooddata)

In [ ]:
detections = segm_map(deep_frame_final, SN=5, return_all=True)

cat = SourceCatalog(deep_frame_final, detections)

create_catalogue(cat)